# []

In [1]:
# inbuilt 
import os
import sys
import math

# most common
import numpy as np
import matplotlib.pyplot as plt

# pytorch
import torch as tt
import torch.nn as nn
import torch.optim as oo
import torch.functional as ff
import torch.distributions as dd
import torch.utils.data as ud

# custom
import known
from known.basic import pj
from known.basic.common import Verbose as verb
import known.ktorch as kt

print(f'{sys.version=}\n{np.__version__=}\n{tt.__version__=}\n{known.__version__=}')

sys.version='3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]'
np.__version__='1.22.2'
tt.__version__='1.10.1+cu102'
known.__version__='0.0.1'


# Select DataSet

In [2]:
seqlen = 24
cols = ('PRICE',)
input_size = len(cols)

ds_test  = kt.SeqDataset.from_csv(pj('data_EVC/normPJDS.csv'), cols=cols, 
                seqlen=seqlen, reverse=True, normalize=False, squeeze_label=True, dtype=tt.float32)
ds_train = kt.SeqDataset.from_csv(pj('data_EVC/train.csv'), cols=cols, 
                seqlen=seqlen, reverse=True, normalize=False, squeeze_label=True, dtype=tt.float32)
ds_val = kt.SeqDataset.from_csv(pj('data_EVC/test.csv'), cols=cols, 
                seqlen=seqlen, reverse=True, normalize=False, squeeze_label=True, dtype=tt.float32)

ds_train, ds_val, ds_test

(<known.ktorch.data.SeqDataset at 0x182c22d4e20>,
 <known.ktorch.data.SeqDataset at 0x182c2299c70>)

# Define Regression Network

In [3]:
#input_size = 3
hidden_size = 64
has_bias = True
dtype=tt.float32
device = None

i2o_sizes = None
o2o_sizes = None
i2o_act = None
o2o_act = None

cells = [
    kt.RNNSACell(kt.ELMANCell(
        input_size=input_size, hidden_size=hidden_size,
        hidden_bias=has_bias, hidden_activation=None, # none for default
        dtype=dtype, device=device,)), #.\
        #build_i2o(size=i2o_sizes[0], bias=has_bias, activation=i2o_act, dtype=dtype, device=device).\
        #build_o2o(size=o2o_sizes[0], bias=has_bias, activation=o2o_act, dtype=dtype, device=device),

    kt.RNNSACell(kt.ELMANCell(
        input_size=hidden_size, hidden_size=hidden_size,
        hidden_bias=has_bias, hidden_activation=None, # none for default
        dtype=dtype, device=device,)), #.\
        #build_i2o(size=i2o_sizes[0], bias=has_bias, activation=i2o_act, dtype=dtype, device=device).\
        #build_o2o(size=o2o_sizes[0], bias=has_bias, activation=o2o_act, dtype=dtype, device=device),
]
coreF = kt.SRNNSA(cells)

bi = False  
return_sequences=False
stack_output=False   
batch_first=True

rnn = kt.GRNNSA(
    coreF, bi=bi, return_sequences=return_sequences, stack_output=stack_output, batch_first=batch_first
)

class RegRNN(nn.Module):
    def __init__(self, rnn) -> None:
        super().__init__()
        self.rnn = rnn
        self.fc = kt.dense_sequential(
            in_dim=(hidden_size*2 if bi else hidden_size), layer_dims=[128, 64, 32], out_dim=1,
            actF=nn.ReLU, actL=None, dtype=dtype, device=device )

    def forward(self, x):
        y = self.rnn(x)
        return self.fc(y)



# (A) Train and Evaluate

In [4]:
model = RegRNN(rnn)
#-------------------------------------------------
epochs = 20
batch_size=32
shuffle=True
validation_freq = int(epochs/10)
criterion=nn.MSELoss()
lr = 0.0025
weight_decay = 0.0
optimizer=oo.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
lrs=oo.lr_scheduler.LinearLR(optimizer, start_factor= 1.0, end_factor=0.7, total_iters=epochs)

#early_stop_train=kt.QuantiyMonitor('TrainLoss', patience=50, delta=0.00001)
#early_stop_val=kt.QuantiyMonitor('ValLoss', patience=50, delta=0.00001)
#checkpoint_freq=int(epochs/4)
save_path=f'rnn'


trainer = kt.Trainer(model)
trainer.optimizer=optimizer
trainer.criterion=criterion

trainer.fit(training_data=ds_train, validation_data=ds_val, 
            epochs=epochs, batch_size=batch_size,shuffle=shuffle,validation_freq=validation_freq,
            save_path=save_path, verbose=1)

loss_plot_start = int(epochs/20)
trainer.plot_results(color='black',loss_plot_start=loss_plot_start)

mtl, _ = trainer.evaluate(ds_test, batch_size=None)
#train_loss = np.array(trainer.train_loss_history)
test_loss = mtl

print(f'{test_loss=}')

Training samples: [6047]
Training batches: [189]
Validation samples: [2568]
Validation batches: [1]
Start Training @ 2023-01-29 06:22:10.509513
-------------------------------------------


KeyboardInterrupt: 

# (B) Load and Evaluate

In [8]:

#kt.load_state(model, f'rnn')

trainer = kt.Trainer(model)
trainer.criterion=nn.MSELoss() 
mtl, tl = trainer.evaluate(ds_test, batch_size=1)

test_loss= mtl
print(f'{test_loss=}')

Testing samples: [8687]
Testing batches: [8687]


KeyboardInterrupt: 

# Manual Testing

## test dataset

In [6]:
model.eval()
with tt.no_grad():
    dl = iter(ds_test.dataloader(batch_size=int(len(ds_test)*1.0)))
    Xv, Yv = next(dl)
    Pv = model(Xv)
key_len = (len(Pv))
print(key_len)

IndexError: index 32 is out of bounds for dimension 0 with size 32

## visualize

In [ ]:
fr = 7000
tr = fr + 400
print(f'{fr} ==> {tr}')

for i in range(input_size):
    plt.figure(figsize=(20,10))
    plt.title(f'{i}')
    
    plt.plot(Yv[fr:tr,i], color='black', label='Truth', linewidth=0.5)
    plt.plot(Pv[fr:tr,i], color='green', label=f'Pred', linewidth=0.5)
    plt.legend()
    plt.show()
    plt.close()
    
    


In [ ]:
ilen = 1000 #int(key_len/4)

start_i = 0 #<--------------set this
end_i = int(key_len*1)
print(ilen, key_len, end_i, '\n')

fr, tr, cnt = (start_i)*ilen, (start_i+1)*ilen, 1
while True:
    print(f'#{cnt} :: {fr} ==> {tr}')
    cnt+=1
    for i in range(input_size):
        plt.figure(figsize=(20,10))
        plt.title(f'{i}')
        
        plt.plot(Yv[fr:tr,i], color='black', label='Truth', linewidth=0.5)
        plt.plot(Pv[fr:tr,i], color='green', label=f'Pred', linewidth=0.5)
        plt.legend()
        plt.show()
        plt.close()
    
    
    
    fr=tr
    if fr>=end_i: break
    tr = min(tr+ilen, end_i)
    if tr>end_i: tr=end_i
    